In [1]:
# Primeiro, a importação das bibliotecas necessárias. 

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [2]:
# Para evitar mensagens de erro do Windows devido o K-Means , será utlizada uma opção da biblioteca os. 
os.environ["OMP_NUM_THREADS"] = "1"

In [3]:
# A descrição da localização do arquivo de dados compilado (Base CVM). 
caminho_origem = r"C:\Users\adaut\Downloads\proj_cred\dados\arq_dados.csv"

df = pd.read_csv(caminho_origem, sep=';', decimal=',', encoding='utf-8')

#Visualizar as 5 primeiras linhas para ter certeza. 
df.head()

,DENOM_CIA,CD_CVM,Receita,Ebit,Lucro,Ativo_total,Ativo_circulante,Caixa,Passivo_circulante,Divida_curto_prazo,Divida_longo_prazo,Divida_liquida,Alavancagem
0,2W ECOBANK S.A. - EM RECUPERAÇÃO JUDICIAL,25224,535601,-41464,-407042,1820195,197260,8363,1434676,582117,1029506,1603260,-38.666313
1,PANATLANTICA S.A.,94,2109725,128411,84659,1617429,1204075,25071,528091,303558,179430,457917,3.566026
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,23396,14212394,5807716,2396851,44331033,8194859,182644,4732844,22770313,2010990,24598659,4.235513
3,BIOMA EDUCAÇÃO S.A.,701,354632,-27063,-74340,528180,51903,15715,164722,45037,11929,41251,-1.524258
4,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,25283,1516484,-741103,-934083,2367614,1062544,345841,2069726,82945,1473872,1210976,-1.634018


In [4]:
#Agora, a Engenharia de Atributos  

# Para o cálculo correto, deve fazer a substituição de valores nulos por 0.
df = df.fillna(0)

# Para fins contabeis serao criadas colunas com informacoes cruciais. 

# Calcular a divida bruta somando a de curto e longo prazo
df['Divida_Total'] = df['Divida_curto_prazo'] + df['Divida_longo_prazo']
# Calcular a dívida real (se todo dinheiro disponível fosse utilizado, o que sobraria?)
df['Divida_Liquida'] = df['Divida_Total'] - df['Caixa']
#Calcular a saúde financeira. Obs.: o np.where é para evitar erro quando Ebit for igual a zero.
df['Alavancagem'] = np.where(df['Ebit'] == 0, 0, df['Divida_Liquida'] / df['Ebit'])
# Calcular a Eficiência (para cada R$ negociado, quanto sobra no bolso?)
df['Margem_Liquida'] = np.where(df['Receita'] == 0, 0, df['Lucro'] / df['Receita'])

#Verificando como ficou...
df.columns

Index(['DENOM_CIA', 'CD_CVM', 'Receita', 'Ebit', 'Lucro', 'Ativo_total',
       'Ativo_circulante', 'Caixa', 'Passivo_circulante', 'Divida_curto_prazo',
       'Divida_longo_prazo', 'Divida_liquida', 'Alavancagem', 'Divida_Total',
       'Divida_Liquida', 'Margem_Liquida'],
      dtype='object')

In [5]:
# Agora sera criado um data frame mais limpo (tirando empresas de holding e outras que nao operam). 
df_clean = df[
    (df['Receita'] > 0) &
    (df['Alavancagem'] > -10) & (df['Alavancagem'] < 20) &
    (df['Margem_Liquida'] > -5) & (df['Margem_Liquida'] < 5)
].copy()


In [6]:
# Consultar quantas empresas restaram das 457 inicias...
len(df_clean)

397

In [7]:
# A utlização do Machine Learning ...

#Primeiro, a padronização dos dados
# AQUI ESTÁ A CORREÇÃO: Criando o X_scaled antes de usar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_clean[['Alavancagem', 'Margem_Liquida']])

# Agora sim, aplicação o K-Means nos dados escalados. 
#n_clusters=4: Define que os dados serão divididos em 4 grupos (clusters).
#random_state=42: Garante a reprodutibilidade. Isso significa que, toda que rodar esse código, os grupos sera identicos. 
# n_init=10: O algoritmo rodará 10 vezes com diferentes sementes iniciais de centroides (início aleatório). 
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_clean['Cluster'] = kmeans.fit_predict(X_scaled)

C:\Users\adaut\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


In [8]:
# Configuração do DBSCAN
# eps=2.0: Raio de busca (2 desvios padrão, já que os dados estão normalizados)
# min_samples=5: Mínimo de empresas para formar um grupo denso
dbscan = DBSCAN(eps=2.0, min_samples=5)
df_clean['Cluster_DBSCAN'] = dbscan.fit_predict(X_scaled)

# Contagem dos grupos encontrados
# Nota: No DBSCAN, o Cluster '-1' representa RUÍDO (Outliers/Anomalias)
print("   > Grupos encontrados pelo DBSCAN:")
print(df_clean['Cluster_DBSCAN'].value_counts())

# Validação Cruzada:
# Ver quantos "Alto Risco" do K-Means também foram marcados como "Outlier" (-1) pelo DBSCAN
outliers_dbscan = df_clean[df_clean['Cluster_DBSCAN'] == -1]
print(f"   > Total de anomalias extremas detectadas: {len(outliers_dbscan)}")

   > Grupos encontrados pelo DBSCAN:
Cluster_DBSCAN
 0    394
-1      3
Name: count, dtype: int64
   > Total de anomalias extremas detectadas: 3


In [9]:
#Por fim, a exportação do arquivo para criar a visualização.  
caminho_final = r"C:\Users\adaut\Downloads\resultado_clusterizacao.csv"
df_clean.to_csv(caminho_final, sep=';', decimal=',', index=False, encoding='utf-8-sig')
